In [1]:
import nltk
from nltk.stem.porter import *
from torch.nn import *
from torch.optim import *
import numpy as np
import pandas as pd
import torch,torchvision
import random
from tqdm import *
from torch.utils.data import Dataset,DataLoader
stemmer = PorterStemmer()

In [2]:
def tokenize(sentence):
    return nltk.word_tokenize(sentence)

In [3]:
tokenize('$1000')

['$', '1000']

In [4]:
def stem(word):
    return stemmer.stem(word.lower())

In [5]:
stem('organic')

'organ'

In [6]:
def bag_of_words(tokenized_words,all_words):
    tokenized_words = [stem(w) for w in tokenized_words]
    bag = np.zeros(len(all_words),dtype=np.float32)
    for idx,w in enumerate(all_words):
        if w in tokenized_words:
            bag[idx] = 1.0
    return bag

In [7]:
bag_of_words(['hi'],['oloegsegse','hi'])

array([0., 1.], dtype=float32)

In [8]:
data = pd.read_csv('./data.csv')

In [9]:
X = data['text']
y = data['label']

In [10]:
all_words = []
all_data = []
tags = []

In [11]:
for X_batch,y_batch in tqdm(zip(X,y)):
    X_batch = tokenize(X_batch)
    new_X = []
    for X_batch_in_X_batch in X_batch:
        new_X.append(stem(X_batch_in_X_batch))
    all_words.extend(new_X)
    all_data.append((new_X,y_batch))
    tags.append(y_batch)

3613it [00:01, 3061.61it/s]


In [12]:
all_words = sorted(set(all_words))
tags = sorted(set(tags))

In [13]:
tags

['anger', 'fear', 'joy', 'sadness']

In [14]:
X = []
y = []

In [15]:
for sentence,tag in all_data:
    sentence = bag_of_words(sentence,all_words)
    X.append(sentence)
    y.append(tags.index(tag))

In [16]:
from sklearn.model_selection import *
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.125,shuffle=False)
device = 'cuda'

In [17]:
X_train = torch.from_numpy(np.array(X_train)).to(device).float()
y_train = torch.from_numpy(np.array(y_train)).to(device).to(device).long()
X_test = torch.from_numpy(np.array(X_test)).to(device).to(device).float()
y_test = torch.from_numpy(np.array(y_test)).to(device).to(device).long()

In [18]:
def get_loss(model,X,y,criterion):
    preds = model(X)
    loss = criterion(preds,y)
    return loss.item()

In [19]:
def get_accuracy(model,X,y):
    correct = 0
    total = 0
    preds = model(X)
    for pred,y_batch in zip(preds,y):
        pred = int(torch.argmax(pred))
        if pred == y_batch:
            correct += 1
        total += 1
    print(correct,total)
    acc = round(correct/total,3)*100
    return acc

In [ ]:
len(tags)

In [20]:
class Model(Module):
    def __init__(self,hidden=2048,iters=12,activation=ReLU):
        super().__init__()
        self.iters = iters
        self.activation = activation()
        self.hidden = hidden
        self.linear1 = Linear(len(all_words),hidden)
        self.linear2 = Linear(hidden,hidden)
        self.output = Linear(hidden,len(tags)+1)
    
    def forward(self,X):
        preds = self.linear1(X)
        for _ in range(self.iters):
            preds = self.activation(self.linear2(preds))
        preds = self.output(preds)
        return preds

In [21]:
model = Model().to(device)

In [22]:
criterion = CrossEntropyLoss()

In [23]:
optimizer = Adam(model.parameters(),lr=0.001)

In [24]:
epochs = 100

In [25]:
batch_size = 32

In [26]:
import wandb

In [ ]:
wandb.init(project='Emotion-Classification-NLP',name='baseline')
wandb.watch(model)
for _ in tqdm(range(epochs)):
    for idx in range(0,len(X_train),batch_size):
        X_batch = X_train[idx:idx+batch_size].to(device)
        y_batch = y_train[idx:idx+batch_size].to(device)
        preds = model(X_batch)
        loss = criterion(preds,y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    wandb.log({'Loss':get_loss(model,X_train,y_train,criterion)})
    wandb.log({'Val Loss':get_loss(model,X_test,y_test,criterion)})
    wandb.log({'Acc':get_accuracy(model,X_train,y_train)})
    wandb.log({'Val Acc':get_accuracy(model,X_test,y_test)})
wandb.watch(model)
wandb.finish()

wandb: Currently logged in as: ranuga-d (use `wandb login --relogin` to force relogin)


  1%|▍                                          | 1/100 [00:01<02:49,  1.71s/it]

857 3161
0 452


  2%|▊                                          | 2/100 [00:03<02:49,  1.73s/it]

1231 3161
0 452


...

 92%|██████████████████████████████████████▋   | 92/100 [02:39<00:13,  1.71s/it]

3112 3161
174 452


 93%|███████████████████████████████████████   | 93/100 [02:41<00:11,  1.70s/it]

3075 3161
188 452


 94%|███████████████████████████████████████▍  | 94/100 [02:42<00:10,  1.70s/it]